In [1]:
# Python3 compatibility
from __future__ import print_function, division, absolute_import
from glob import glob
import numpy as np
import os
home = os.path.expanduser("~")+"/"

import matplotlib.pyplot as plt

# Load DmpSoftware and ROOT
from ROOT import gSystem
gSystem.Load('libDmpEvent.so')
from ROOT import DmpChain
from ROOT import TChain

In [2]:
plt.style.use(home+"Tools/matplotlib_style")

IOError: '/home/users/c/coppinp/Tools/matplotlib_style' not found in the style library and input is not a valid URL or path; see `style.available` for list of available styles

In [3]:
Use_data = False
if( Use_data ):
    infile = "/dpnc/beegfs/dampe/prod/FM/skim/6.0.0/v2/2018/03/01_data_100_500.root"
    infile = home + "data_file_3.root"
    dc = DmpChain("CollectionTree")
    ttt = dc.Add(infile)
else:
    infile = "/dpnc/beegfs/dampe/prod/MC/reco/v6r0p12/allHe4-v6r0p12_10TeV_100TeV-FTFP_BGO_Quenching/allHe4-v6r0p12_10TeV_100TeV-FTFP_BGO_Quenching.noOrb.500005.reco.root"
    dc = DmpChain("CollectionTree")
    ttt = dc.Add(infile)

nevents = dc.GetEntries()
print(nevents)

1000


TStreamerInfo::BuildCheck:0: RuntimeWarning: 
   The StreamerInfo of class DmpRunSimuHeader read from file /dpnc/beegfs/dampe/prod/MC/reco/v6r0p12/allHe4-v6r0p12_10TeV_100TeV-FTFP_BGO_Quenching/allHe4-v6r0p12_10TeV_100TeV-FTFP_BGO_Quenching.noOrb.500005.reco.root
   has the same version (=1) as the active class but a different checksum.
   You should update the version to ClassDef(DmpRunSimuHeader,2).
   Do not try to write objects with the current class definition,
   the files will not be readable.

TStreamerInfo::CompareContent:0: RuntimeWarning: The following data member of
the in-memory layout version 1 of class 'DmpRunSimuHeader' is missing from 
the on-file layout version 1:
   string fHadrPhysList; //
TStreamerInfo::CompareContent:0: RuntimeWarning: The following data member of
the in-memory layout version 1 of class 'DmpRunSimuHeader' is missing from 
the on-file layout version 1:
   double fCrmcEnergyThresholdGev; //
TStreamerInfo::CompareContent:0: RuntimeWarning: The follow

In [4]:
skim = np.zeros(nevents, dtype=np.bool)
for i in range(nevents):
    ev = dc.GetDmpEvent(i)

    # 1: BGO reconstructed energy greater than 20 GeV
    BGOrec = ev.pEvtBgoRec()
    BGO_TotalE = BGOrec.GetTotalEnergy() # [MeV]
    if( BGO_TotalE<2e4 ):
        continue

    # 2: rMaxELayer cut
    BGO_MaxELayer = max( [BGOrec.GetELayer(j) for j in range(14)] )
    ratio = BGO_MaxELayer/BGO_TotalE
    if( ratio>0.35 ):
        continue

    # 3: iBarMaxE cut
    #   Events where the maximum energy OF A SINGLE HIT is deposited at the edge of the calorimeter?
    #   By 1, 2, 3    So how come we skip the first layer of BGO? --> David: Would prob be removed by vertex cut anyway
    BGOhits = ev.pEvtBgoHits()
    BGO_NHits = BGOhits.GetHittedBarNumber()
    BGO_HitE = np.array(BGOhits.fEnergy, dtype=np.float)
    BGO_HitBarId = np.array(BGOhits.fGlobalBarID, dtype=np.uint16)

    Hit123_maxE = np.zeros(3)
    Hit123_BarIdx = -1*np.ones(3)
    for j in range(BGO_NHits):
        HitE = BGO_HitE[j]
        HitLayer = BGOhits.GetLayerID(j)
        if( (HitLayer in [1,2,3]) and (HitE>Hit123_maxE[HitLayer-1]) ):
            IdxBar = np.divide(BGO_HitBarId[j],2**6)%31
            Hit123_maxE[HitLayer-1] = HitE
            Hit123_BarIdx[HitLayer-1] = IdxBar
    for j in range(3):
        if( Hit123_BarIdx[j]<=0 or Hit123_BarIdx[j]==21 ):
            continue

    # 4 & 5: FullBGO cut (+ shower vertex/direction successfully reconstructed)
    BGO_ZTop, BGO_ZBottom = 46, 448
    BGO_Slope = [BGOrec.GetSlopeYZ(), BGOrec.GetSlopeXZ()]
    BGO_Intercept = [BGOrec.GetInterceptYZ(), BGOrec.GetInterceptXZ()]
    if( (BGO_Slope[1]==0 and BGO_Intercept[1]==0) or (BGO_Slope[0]==0 and BGO_Intercept[0]==0) ):
        continue
    XTop = BGO_Slope[1]*BGO_ZTop + BGO_Intercept[1]
    YTop = BGO_Slope[0]*BGO_ZTop + BGO_Intercept[0]
    XBottom = BGO_Slope[1]*BGO_ZBottom + BGO_Intercept[1]
    YBottom = BGO_Slope[0]*BGO_ZBottom + BGO_Intercept[0]
    if( np.any(np.abs([XTop,YTop,XBottom,YBottom])>280) ):
        continue

    # If we were not stopped by now, set the skim variable to true
    skim[i] = True

In [5]:
print( "Consistency check, are they events in skim selecting that do not pass the filter?")
print("\t", any( ~skim ))

Consistency check, are they events in skim selecting that do not pass the filter?
	 True


In [7]:
np.sum(skim), len(skim)

(13, 1000)

In [4]:
import ROOT

In [ ]:
ROOT.DmpChain